In [1]:
import pandas as pd
import re
import warnings
warnings.simplefilter("ignore")

In [2]:
videogames = pd.read_csv("../inputs/metacritic_games.csv")

In [3]:
videogames

,game,platform,developer,genre,number_players,rating,release_date,positive_critics,neutral_critics,negative_critics,positive_users,neutral_users,negative_users,metascore,user_score
0,Portal 2,PC,Valve Software,Action,NaN,E10+,"Apr 18, 2011",51,1,0,1700,107,19,95,90
1,The Elder Scrolls V: Skyrim,PC,Bethesda Game Studios,Role-Playing,No Online Multiplayer,M,"Nov 10, 2011",32,0,0,1616,322,451,94,82
2,The Legend of Zelda: Ocarina of Time 3D,3DS,GREZZO,Miscellaneous,No Online Multiplayer,E10+,"Jun 19, 2011",84,1,0,283,20,5,94,90
3,Batman: Arkham City,PC,Rocksteady Studios,Action Adventure,NaN,T,"Nov 21, 2011",27,0,0,240,34,27,91,87
4,Super Mario 3D Land,3DS,Nintendo,Action,No Online Multiplayer,E,"Nov 13, 2011",81,1,0,251,39,11,90,84
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5694,Modern Combat: Blackout,Switch,Gameloft,Action,No Online Multiplayer,NaN,"Feb 14, 2019",0,5,2,1,0,1,53,57
5695,Fimbul,PS4,Zaxis,Action Adventure,No Online Multiplayer,NaN,"Feb 28, 2019",0,7,2,1,0,0,52,18
5696,Generation Zero,PC,Avalanche Studios,Action Adventure,Online Multiplayer,T,"Mar 26, 2019",1,11,6,5,2,6,50,57
5697,Vane,PS4,Friend & Foe,Adventure,No Online Multiplayer,E10+,"Jan 15, 2019",3,26,22,2,2,2,49,55


In [4]:
videogames.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5699 entries, 0 to 5698
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   game              5699 non-null   object
 1   platform          5699 non-null   object
 2   developer         5685 non-null   object
 3   genre             5694 non-null   object
 4   number_players    3059 non-null   object
 5   rating            4433 non-null   object
 6   release_date      5699 non-null   object
 7   positive_critics  5699 non-null   int64 
 8   neutral_critics   5699 non-null   int64 
 9   negative_critics  5699 non-null   int64 
 10  positive_users    5699 non-null   int64 
 11  neutral_users     5699 non-null   int64 
 12  negative_users    5699 non-null   int64 
 13  metascore         5699 non-null   int64 
 14  user_score        5699 non-null   int64 
dtypes: int64(8), object(7)
memory usage: 668.0+ KB


In [5]:
videogames.duplicated().value_counts()

False    5699
dtype: int64

In [6]:
videogames.columns

Index(['game', 'platform', 'developer', 'genre', 'number_players', 'rating',
       'release_date', 'positive_critics', 'neutral_critics',
       'negative_critics', 'positive_users', 'neutral_users', 'negative_users',
       'metascore', 'user_score'],
      dtype='object')

In [7]:
videogames1 = videogames[["game","platform","genre","release_date","metascore"]]
videogames1

,game,platform,genre,release_date,metascore
0,Portal 2,PC,Action,"Apr 18, 2011",95
1,The Elder Scrolls V: Skyrim,PC,Role-Playing,"Nov 10, 2011",94
2,The Legend of Zelda: Ocarina of Time 3D,3DS,Miscellaneous,"Jun 19, 2011",94
3,Batman: Arkham City,PC,Action Adventure,"Nov 21, 2011",91
4,Super Mario 3D Land,3DS,Action,"Nov 13, 2011",90
...,...,...,...,...,...
5694,Modern Combat: Blackout,Switch,Action,"Feb 14, 2019",53
5695,Fimbul,PS4,Action Adventure,"Feb 28, 2019",52
5696,Generation Zero,PC,Action Adventure,"Mar 26, 2019",50
5697,Vane,PS4,Adventure,"Jan 15, 2019",49


In [8]:
videogames1.head(10).sort_values(by=["metascore"], ascending=False).reset_index(drop=True)

,game,platform,genre,release_date,metascore
0,Portal 2,PC,Action,"Apr 18, 2011",95
1,The Elder Scrolls V: Skyrim,PC,Role-Playing,"Nov 10, 2011",94
2,The Legend of Zelda: Ocarina of Time 3D,3DS,Miscellaneous,"Jun 19, 2011",94
3,Batman: Arkham City,PC,Action Adventure,"Nov 21, 2011",91
4,Super Mario 3D Land,3DS,Action,"Nov 13, 2011",90
5,Deus Ex: Human Revolution,PC,Action,"Aug 23, 2011",90
6,Pushmo,3DS,Miscellaneous,"Dec 8, 2011",90
7,Total War: Shogun 2,PC,Strategy,"Mar 15, 2011",90
8,FIFA Soccer 12,PC,Sports,"Sep 27, 2011",89
9,Battlefield 3,PC,Action,"Oct 25, 2011",89


In [9]:
videogames2 = videogames1[videogames1["platform"]=="3DS"]
videogames3 = videogames1[videogames1["platform"]== "WIIU"]
videogames4 = videogames1[videogames1["platform"]== "Switch"]
videogames4 = pd.concat([videogames2, videogames3, videogames4], axis= 0)
display(videogames4)

,game,platform,genre,release_date,metascore
2,The Legend of Zelda: Ocarina of Time 3D,3DS,Miscellaneous,"Jun 19, 2011",94
4,Super Mario 3D Land,3DS,Action,"Nov 13, 2011",90
6,Pushmo,3DS,Miscellaneous,"Dec 8, 2011",90
18,Super Street Fighter IV: 3D Edition,3DS,Action,"Mar 27, 2011",85
20,Mario Kart 7,3DS,Driving,"Dec 4, 2011",85
...,...,...,...,...,...
5676,YIIK: A Postmodern RPG,Switch,Role-Playing,"Jan 17, 2019",64
5677,Claybook,Switch,Puzzle,"Mar 12, 2019",63
5680,Sphinx and the Cursed Mummy,Switch,Action Adventure,"Jan 29, 2019",62
5687,The Caligula Effect: Overdose,Switch,Role-Playing,"Mar 12, 2019",59


In [10]:
videogames5 = videogames4[videogames4["metascore"]>70]
videogames5

,game,platform,genre,release_date,metascore
2,The Legend of Zelda: Ocarina of Time 3D,3DS,Miscellaneous,"Jun 19, 2011",94
4,Super Mario 3D Land,3DS,Action,"Nov 13, 2011",90
6,Pushmo,3DS,Miscellaneous,"Dec 8, 2011",90
18,Super Street Fighter IV: 3D Edition,3DS,Action,"Mar 27, 2011",85
20,Mario Kart 7,3DS,Driving,"Dec 4, 2011",85
...,...,...,...,...,...
5612,X-Morph: Defense,Switch,Strategy,"Feb 21, 2019",75
5624,Aragami: Shadow Edition,Switch,Action Adventure,"Feb 21, 2019",74
5633,The Liar Princess and the Blind Prince,Switch,Action,"Feb 12, 2019",72
5641,Chocobo's Mystery Dungeon: Every Buddy!,Switch,General,"Mar 20, 2019",71


In [11]:
videogames5.release_date = videogames5.release_date.str.split(",",).str[1]
videogames5

,game,platform,genre,release_date,metascore
2,The Legend of Zelda: Ocarina of Time 3D,3DS,Miscellaneous,2011,94
4,Super Mario 3D Land,3DS,Action,2011,90
6,Pushmo,3DS,Miscellaneous,2011,90
18,Super Street Fighter IV: 3D Edition,3DS,Action,2011,85
20,Mario Kart 7,3DS,Driving,2011,85
...,...,...,...,...,...
5612,X-Morph: Defense,Switch,Strategy,2019,75
5624,Aragami: Shadow Edition,Switch,Action Adventure,2019,74
5633,The Liar Princess and the Blind Prince,Switch,Action,2019,72
5641,Chocobo's Mystery Dungeon: Every Buddy!,Switch,General,2019,71


In [12]:
videogames5.reset_index(drop=True)

,game,platform,genre,release_date,metascore
0,The Legend of Zelda: Ocarina of Time 3D,3DS,Miscellaneous,2011,94
1,Super Mario 3D Land,3DS,Action,2011,90
2,Pushmo,3DS,Miscellaneous,2011,90
3,Super Street Fighter IV: 3D Edition,3DS,Action,2011,85
4,Mario Kart 7,3DS,Driving,2011,85
...,...,...,...,...,...
695,X-Morph: Defense,Switch,Strategy,2019,75
696,Aragami: Shadow Edition,Switch,Action Adventure,2019,74
697,The Liar Princess and the Blind Prince,Switch,Action,2019,72
698,Chocobo's Mystery Dungeon: Every Buddy!,Switch,General,2019,71


In [13]:
apiDF = pd.read_csv("../outputs/api_clean_metacritic_games.csv")

In [14]:
apiDF = apiDF.drop(['Unnamed: 0'], axis=1)

In [15]:
apiDF.reset_index(drop=True).head()

,game,platform,genre,release_date,metascore
0,Animal Crossing: New Horizons,Switch,Simulation,2020,90
1,Xenoblade Chronicles: Definitive Edition,Switch,RPG,2020,89
2,DOOM Eternal,Switch,Action,2020,88
3,Risk of Rain 2,Switch,Action,2020,85
4,Streets of Rage 4,Switch,Casual,2020,84


In [16]:
clean_metacritic_games = pd.concat([videogames5, apiDF], axis=0)

In [17]:
clean_metacritic_games.platform.value_counts()

Switch    406
3DS       208
WIIU      121
Name: platform, dtype: int64

In [18]:
clean_metacritic_games.reset_index(drop=True)

,game,platform,genre,release_date,metascore
0,The Legend of Zelda: Ocarina of Time 3D,3DS,Miscellaneous,2011,94
1,Super Mario 3D Land,3DS,Action,2011,90
2,Pushmo,3DS,Miscellaneous,2011,90
3,Super Street Fighter IV: 3D Edition,3DS,Action,2011,85
4,Mario Kart 7,3DS,Driving,2011,85
...,...,...,...,...,...
730,Prison Princess,Switch,Casual,2020,73
731,Cloudpunk,Switch,Casual,2020,73
732,Captain Tsubasa: Rise of New Champions,Switch,Casual,2020,73
733,Biped,Switch,Adventure,2020,72


In [19]:
clean_metacritic_games.to_csv("../outputs/clean_metacritic_games.csv")